In [1]:
from sklearn.metrics import roc_curve
def ks_metric(true,score):
    fpr, tpr, thresholds = roc_curve(true,score)
    ks = max(fpr-tpr)
    return ks

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing
import warnings

warnings.filterwarnings("ignore")

import time


def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt


def base_process(data):
    lbl = preprocessing.LabelEncoder()
    print(
        '--------------------------------------------------------------item--------------------------------------------------------------')
    data['len_item_category'] = data['item_category_list'].map(lambda x: len(str(x).split(';')))
    data['len_item_property'] = data['item_property_list'].map(lambda x: len(str(x).split(';')))
    for i in range(1, 3):
        data['item_category_list' + str(i)] = lbl.fit_transform(data['item_category_list'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))  # item_category_list的第0列全部都一样
    for i in range(10):
        data['item_property_list' + str(i)] = lbl.fit_transform(data['item_property_list'].map(lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
    for col in ['item_id', 'item_brand_id', 'item_city_id']:
        data[col] = lbl.fit_transform(data[col])
    print(
        '--------------------------------------------------------------user--------------------------------------------------------------')
    for col in ['user_id']:
        data[col] = lbl.fit_transform(data[col])
    print('user 0,1 feature')
    data['gender0'] = data['user_gender_id'].apply(lambda x: 1 if x == -1 else 2)
    data['age0'] = data['user_age_level'].apply(lambda x: 1 if x == 1004 | x == 1005 | x == 1006 | x == 1007  else 2)
    data['occupation0'] = data['user_occupation_id'].apply(lambda x: 1 if x == -1 | x == 2003  else 2)
    data['star0'] = data['user_star_level'].apply(lambda x: 1 if x == -1 | x == 3000 | x == 3001  else 2)
    print(
        '--------------------------------------------------------------context--------------------------------------------------------------')
    data['realtime'] = data['context_timestamp'].apply(timestamp_datetime)
    data['realtime'] = pd.to_datetime(data['realtime'])
    data['day'] = data['realtime'].dt.day
    data['hour'] = data['realtime'].dt.hour
    data['len_predict_category_property'] = data['predict_category_property'].map(lambda x: len(str(x).split(';')))
    for i in range(5):
        data['predict_category_property' + str(i)] = lbl.fit_transform(data['predict_category_property'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))
    print('context 0,1 feature')
    data['context_page0'] = data['context_page_id'].apply(
        lambda x: 1 if x == 4001 | x == 4002 | x == 4003 | x == 4004 | x == 4007  else 2)
    print(
        '--------------------------------------------------------------shop--------------------------------------------------------------')
    for col in ['shop_id']:
        data[col] = lbl.fit_transform(data[col])
    data['shop_score_delivery0'] = data['shop_score_delivery'].apply(lambda x: 0 if x <= 0.98 and x >= 0.96  else 1)
    return data


def map_hour(x):
    if (x>=7)&(x<=12):
        return 1
    elif (x>=13)&(x<=20):
        return 2
    else:
        return 3

def deliver(x):
    #x=round(x,6)
    jiange=0.1
    for i in range(1,20):
        if (x>=4.1+jiange*(i-1))&(x<=4.1+jiange*i):
            return i+1
    if x==-5:
        return 1

def deliver1(x):
    if (x>=2)&(x<=4):
        return 1
    elif (x>=5)&(x<=7):
        return 2
    else:
        return 3


def review(x):
    # x=round(x,6)
    jiange = 0.02
    for i in range(1, 30):
        if (x >= 0.714 + jiange * (i - 1)) & (x <= 0.714 + jiange * i):
            return i + 1
    if x == -1:
        return 1

def review1(x):
    # x=round(x,6)
    if (x>=2)&(x<=12):
        return 1
    elif (x>=13)&(x<=15):
        return 2
    else:
        return 3


def service(x):
    #x=round(x,6)
    jiange=0.1
    for i in range(1,20):
        if (x>=3.93+jiange*(i-1))&(x<=3.93+jiange*i):
            return i+1
    if x==-1:
        return 1

def service1(x):
    if (x>=2)&(x<=7):
        return 1
    elif (x>=8)&(x<=9):
        return 2
    else:
        return 3


def describe(x):
    #x=round(x,6)
    jiange=0.1
    for i in range(1,30):
        if (x>=3.93+jiange*(i-1))&(x<=3.93+jiange*i):
            return i+1
    if x==-1:
        return 1

def describe1(x):
    if (x>=2)&(x<=8):
        return 1
    elif (x>=9)&(x<=10):
        return 2
    else:
        return 3

def shijian(data):
    data['hour_map'] = data['hour'].apply(map_hour)
    return data

def shop_fenduan(data):
    data['shop_score_delivery'] = data['shop_score_delivery'] * 5
    data = data[data['shop_score_delivery'] != -5]
    data['deliver_map'] = data['shop_score_delivery'].apply(deliver)
    data['deliver_map'] = data['deliver_map'].apply(deliver1)
    # del data['shop_score_delivery']
    print(data.deliver_map.value_counts())

    data['shop_score_service'] = data['shop_score_service'] * 5
    data = data[data['shop_score_service'] != -5]
    data['service_map'] = data['shop_score_service'].apply(service)
    data['service_map'] = data['service_map'].apply(service1)
    # del data['shop_score_service']
    print(data.service_map.value_counts())  # 视为好评，中评，差评
    #
    data['shop_score_description'] = data['shop_score_description'] * 5
    data = data[data['shop_score_description'] != -5]
    data['de_map'] = data['shop_score_description'].apply(describe)
    data['de_map'] = data['de_map'].apply(describe1)
    # del data['shop_score_description']
    print(data.de_map.value_counts())

    data = data[data['shop_review_positive_rate'] != -1]
    data['review_map'] = data['shop_review_positive_rate'].apply(review)
    data['review_map'] = data['review_map'].apply(review1)
    print(data.review_map.value_counts())

    data['normal_shop'] = data.apply(
        lambda x: 1 if (x.deliver_map == 3) & (x.service_map == 3) & (x.de_map == 3) & (x.review_map == 3) else 0,
        axis=1)
    del data['de_map']
    del data['service_map']
    del data['deliver_map']
    del data['review_map']
    return data


def slide_cnt(data):
    # item_cnt = data.groupby(by='item_id').count()['instance_id'].to_dict()
    # data['item_cnt'] = data['item_id'].apply(lambda x: item_cnt[x])
    # user_cnt = data.groupby(by='user_id').count()['instance_id'].to_dict()
    # data['user_cnt'] = data['user_id'].apply(lambda x: user_cnt[x])
    # shop_cnt = data.groupby(by='shop_id').count()['instance_id'].to_dict()
    # data['shop_cnt'] = data['shop_id'].apply(lambda x: shop_cnt[x])

    print('当前日期前一天的cnt')
    for d in range(19, 26):  # 18到24号
        df1 = data[data['day'] == d - 1]
        df2 = data[data['day'] == d]  # 19到25号
        user_cnt = df1.groupby(by='user_id').count()['instance_id'].to_dict()
        item_cnt = df1.groupby(by='item_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        df2['user_cnt1'] = df2['user_id'].apply(lambda x: user_cnt.get(x, 0))
        df2['item_cnt1'] = df2['item_id'].apply(lambda x: item_cnt.get(x, 0))
        df2['shop_cnt1'] = df2['shop_id'].apply(lambda x: shop_cnt.get(x, 0))
        df2 = df2[['user_cnt1', 'item_cnt1', 'shop_cnt1', 'instance_id']]
        if d == 19:
            Df2 = df2
        else:
            Df2 = pd.concat([df2, Df2])
    data = pd.merge(data, Df2, on=['instance_id'], how='left')
    print('当前日期之前的cnt')
    for d in range(19, 26):
        # 19到25，25是test
        df1 = data[data['day'] < d]
        df2 = data[data['day'] == d]
        user_cnt = df1.groupby(by='user_id').count()['instance_id'].to_dict()
        item_cnt = df1.groupby(by='item_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        df2['user_cntx'] = df2['user_id'].apply(lambda x: user_cnt.get(x, 0))
        df2['item_cntx'] = df2['item_id'].apply(lambda x: item_cnt.get(x, 0))
        df2['shop_cntx'] = df2['shop_id'].apply(lambda x: shop_cnt.get(x, 0))
        df2 = df2[['user_cntx', 'item_cntx', 'shop_cntx', 'instance_id']]
        if d == 19:
            Df2 = df2
        else:
            Df2 = pd.concat([df2, Df2])
    data = pd.merge(data, Df2, on=['instance_id'], how='left')

    print("前一个小时的统计量")

    return data


def zuhe(data):
    for col in ['user_gender_id','user_age_level','user_occupation_id','user_star_level']:
        data[col] = data[col].apply(lambda x: 0 if x == -1 else x)

    for col in ['item_sales_level', 'item_price_level', 'item_collected_level',
                'user_gender_id','user_age_level','user_occupation_id','user_star_level',
                'shop_review_num_level', 'shop_star_level']:
        data[col] = data[col].astype(str)

    print('item两两组合')
    data['sale_price'] = data['item_sales_level'] + data['item_price_level']
    data['sale_collect'] = data['item_sales_level'] + data['item_collected_level']
    data['price_collect'] = data['item_price_level'] + data['item_collected_level']

    print('user两两组合')
    data['gender_age'] = data['user_gender_id'] + data['user_age_level']
    data['gender_occ'] = data['user_gender_id'] + data['user_occupation_id']
    data['gender_star'] = data['user_gender_id'] + data['user_star_level']

    print('shop两两组合')
    data['review_star'] = data['shop_review_num_level'] + data['shop_star_level']


    for col in ['item_sales_level', 'item_price_level', 'item_collected_level',  'sale_price','sale_collect', 'price_collect',
                'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level','gender_age','gender_occ','gender_star',
                'shop_review_num_level','shop_star_level','review_star']:
        data[col] = data[col].astype(int)

    del data['review_star']

    return data

def item(data):
    print('一个item有多少brand,price salse collected level……')

    itemcnt = data.groupby(['item_id'], as_index=False)['instance_id'].agg({'item_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_id'], how='left')

    for col in ['item_brand_id','item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_id'], as_index=False)['instance_id'].agg({str(col) + '_item_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_id'], how='left')
        data[str(col) + '_item_prob']=data[str(col) + '_item_cnt']/data['item_cnt']
    del data['item_cnt']

    print('一个brand有多少price salse collected level……')

    itemcnt = data.groupby(['item_brand_id'], as_index=False)['instance_id'].agg({'item_brand_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_brand_id'], how='left')

    for col in ['item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_brand_id'], as_index=False)['instance_id'].agg({str(col) + '_brand_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_brand_id'], how='left')
        data[str(col) + '_brand_prob'] = data[str(col) + '_brand_cnt'] / data['item_brand_cnt']
    del data['item_brand_cnt']

    print('一个city有多少item_price_level，item_sales_level，item_collected_level，item_pv_level')

    itemcnt = data.groupby(['item_city_id'], as_index=False)['instance_id'].agg({'item_city_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_city_id'], how='left')
    for col in ['item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_city_id'], as_index=False)['instance_id'].agg({str(col) + '_city_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_city_id'], how='left')
        data[str(col) + '_city_prob'] = data[str(col) + '_city_cnt'] / data['item_city_cnt']
    del data['item_city_cnt']

    print('一个price有多少item_sales_level，item_collected_level，item_pv_level')

    itemcnt = data.groupby(['item_price_level'], as_index=False)['instance_id'].agg({'item_price_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_price_level'], how='left')
    for col in ['item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_city_id'], as_index=False)['instance_id'].agg({str(col) + '_price_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_city_id'], how='left')
        data[str(col) + '_price_prob'] = data[str(col) + '_price_cnt'] / data['item_price_cnt']
    del data['item_price_cnt']

    print('一个item_sales_level有多少item_collected_level，item_pv_level')

    itemcnt = data.groupby(['item_sales_level'], as_index=False)['instance_id'].agg({'item_salse_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_sales_level'], how='left')
    for col in ['item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby([col, 'item_sales_level'], as_index=False)['instance_id'].agg({str(col) + '_salse_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_sales_level'], how='left')
        data[str(col) + '_salse_prob'] = data[str(col) + '_salse_cnt'] / data['item_salse_cnt']
    del data['item_salse_cnt']

    print('一个item_collected_level有多少item_pv_level')

    itemcnt = data.groupby(['item_collected_level'], as_index=False)['instance_id'].agg({'item_coll_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['item_collected_level'], how='left')
    for col in ['item_pv_level']:
        itemcnt = data.groupby([col, 'item_collected_level'], as_index=False)['instance_id'].agg({str(col) + '_coll_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'item_collected_level'], how='left')
        data[str(col) + '_coll_prob'] = data[str(col) + '_coll_cnt'] / data['item_coll_cnt']
    del data['item_coll_cnt']

    return data

def user(data):
    print('用户有多少性别')
    itemcnt = data.groupby(['user_id'], as_index=False)['instance_id'].agg({'user_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_id'], how='left')

    for col in ['user_gender_id','user_age_level', 'user_occupation_id', 'user_star_level']:
        itemcnt = data.groupby([col, 'user_id'], as_index=False)['instance_id'].agg({str(col) + '_user_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob']=data[str(col) + '_user_cnt']/data['user_cnt']
    del data['user_cnt']

    print('性别的年龄段，职业有多少')
    itemcnt = data.groupby(['user_gender_id'], as_index=False)['instance_id'].agg({'user_gender_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_gender_id'], how='left')

    for col in ['user_age_level', 'user_occupation_id', 'user_star_level']:
        itemcnt = data.groupby([col, 'user_gender_id'], as_index=False)['instance_id'].agg({str(col) + '_user_gender_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob']=data[str(col) + '_user_gender_cnt']/data['user_gender_cnt']
    del data['user_gender_cnt']

    print('user_age_level对应的user_occupation_id，user_star_level')
    itemcnt = data.groupby(['user_age_level'], as_index=False)['instance_id'].agg({'user_age_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_age_level'], how='left')

    for col in ['user_occupation_id', 'user_star_level']:
        itemcnt = data.groupby([col, 'user_age_level'], as_index=False)['instance_id'].agg({str(col) + '_user_age_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob']=data[str(col) + '_user_age_cnt']/data['user_age_cnt']
    del data['user_age_cnt']

    print('user_occupation_id对应的user_star_level')
    itemcnt = data.groupby(['user_occupation_id'], as_index=False)['instance_id'].agg({'user_occ_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_occupation_id'], how='left')
    for col in ['user_star_level']:
        itemcnt = data.groupby([col, 'user_occupation_id'], as_index=False)['instance_id'].agg({str(col) + '_user_occ_cnt': 'count'})
        data = pd.merge(data, itemcnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob']=data[str(col) + '_user_occ_cnt']/data['user_occ_cnt']
    del data['user_occ_cnt']

    return data

def user_item(data):
    itemcnt = data.groupby(['user_id'], as_index=False)['instance_id'].agg({'user_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_id'], how='left')
    print('一个user有多少item_id,item_brand_id……')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'user_id'], as_index=False)['instance_id'].agg({str(col)+'_user_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob'] = data[str(col) + '_user_cnt'] / data['user_cnt']

    print('一个user_gender有多少item_id,item_brand_id……')
    itemcnt = data.groupby(['user_gender_id'], as_index=False)['instance_id'].agg({'user_gender_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_gender_id'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'user_gender_id'], as_index=False)['instance_id'].agg({str(col)+'_user_gender_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob'] = data[str(col) + '_user_gender_cnt'] / data['user_gender_cnt']

    print('一个user_age_level有多少item_id,item_brand_id……')
    itemcnt = data.groupby(['user_age_level'], as_index=False)['instance_id'].agg({'user_age_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_age_level'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'user_age_level'], as_index=False)['instance_id'].agg({str(col)+'_user_age_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob'] = data[str(col) + '_user_age_cnt'] / data['user_age_cnt']

    print('一个user_occupation_id有多少item_id,item_brand_id…')
    itemcnt = data.groupby(['user_occupation_id'], as_index=False)['instance_id'].agg({'user_occ_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['user_occupation_id'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'user_occupation_id'], as_index=False)['instance_id'].agg({str(col)+'_user_occ_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob'] = data[str(col) + '_user_occ_cnt'] / data['user_occ_cnt']

    return data


def user_shop(data):
    print('一个user有多少shop_id,shop_review_num_level……')

    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby([col, 'user_id'], as_index=False)['instance_id'].agg(
            {str(col) + '_user_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob'] = data[str(col) + '_user_cnt'] / data['user_cnt']
    del data['user_cnt']

    print('一个user_gender有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby([col, 'user_gender_id'], as_index=False)['instance_id'].agg(
            {str(col) + '_user_gender_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob'] = data[str(col) + '_user_gender_cnt'] / data['user_gender_cnt']
    del data['user_gender_cnt']

    print('一个user_age_level有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby([col, 'user_age_level'], as_index=False)['instance_id'].agg(
            {str(col) + '_user_age_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob'] = data[str(col) + '_user_age_cnt'] / data['user_age_cnt']
    del data['user_age_cnt']

    print('一个user_occupation_id有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby([col, 'user_occupation_id'], as_index=False)['instance_id'].agg(
            {str(col) + '_user_occ_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob'] = data[str(col) + '_user_occ_cnt'] / data['user_occ_cnt']
    del data['user_occ_cnt']

    return data


def shop_item(data):
    print('一个shop有多少item_id,item_brand_id,item_city_id,item_price_level……')
    itemcnt = data.groupby(['shop_id'], as_index=False)['instance_id'].agg({'shop_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['shop_id'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'shop_id'], as_index=False)['instance_id'].agg({str(col)+'_shop_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'shop_id'], how='left')
        data[str(col) + '_shop_prob'] = data[str(col) + '_shop_cnt'] / data['shop_cnt']
    del data['shop_cnt']

    print('一个shop_review_num_level有多少item_id,item_brand_id,item_city_id,item_price_level……')
    itemcnt = data.groupby(['shop_review_num_level'], as_index=False)['instance_id'].agg({'shop_rev_cnt': 'count'})
    data = pd.merge(data, itemcnt, on=['shop_review_num_level'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = data.groupby([col, 'shop_review_num_level'], as_index=False)['instance_id'].agg({str(col)+'_shop_rev_cnt': 'count'})
        data = pd.merge(data, item_shop_cnt, on=[col, 'shop_review_num_level'], how='left')
        data[str(col) + '_shop_rev_prob'] = data[str(col) + '_shop_rev_cnt'] / data['shop_rev_cnt']
    del data['shop_rev_cnt']

    # print('一个shop_star_level有多少item_id,item_brand_id,item_city_id,item_price_level……')
    # itemcnt = data.groupby(['shop_star_level'], as_index=False)['instance_id'].agg({'shop_star_cnt': 'count'})
    # data = pd.merge(data, itemcnt, on=['shop_star_level'], how='left')
    # for col in ['item_id',
    #             'item_brand_id', 'item_city_id', 'item_price_level',
    #             'item_sales_level', 'item_collected_level', 'item_pv_level']:
    #     item_shop_cnt = data.groupby([col, 'shop_star_level'], as_index=False)['instance_id'].agg({str(col) + '_shop_star_cnt': 'count'})
    #     data = pd.merge(data, item_shop_cnt, on=[col, 'shop_star_level'], how='left')
    #     data[str(col) + '_shop_star_prob'] = data[str(col) + '_shop_star_cnt'] / data['shop_star_cnt']
    # del data['shop_star_cnt']
    return data

In [3]:
def lgbCV(train, test,leaf=35):
    col = [c for c in train if
           c not in ['is_trade', 'item_category_list', 'item_property_list', 'predict_category_property', 'instance_id',
                     'context_id', 'realtime', 'context_timestamp']]
    # cat = ['sale_price', 'gender_star', 'user_age_level', 'item_price_level', 'item_sales_level', 'sale_collect',
    #        'price_collect', 'item_brand_id', 'user_star_level', 'item_id', 'shop_id',
    #        'item_city_id', 'context_page_id', 'gender_age', 'shop_star_level', 'item_pv_level', 'user_occupation_id',
    #        'day', 'gender_occ', 'user_gender_id']
    X = train[col]
    y = train['is_trade'].values
    X_tes = test[col]
    y_tes = test['is_trade'].values
    print('Training LGBM model...')
    lgb0 = lgb.LGBMClassifier(
        objective='binary',
        # metric='binary_error',
        num_leaves=leaf,
        depth=8,
        learning_rate=0.05,
        seed=2018,
        colsample_bytree=0.8,
        # min_child_samples=8,
        subsample=0.9,
        n_estimators=20000)
    lgb_model = lgb0.fit(X, y, eval_set=[(X_tes, y_tes)], early_stopping_rounds=200)
    best_iter = lgb_model.best_iteration_
    predictors = [i for i in X.columns]
    feat_imp = pd.Series(lgb_model.feature_importances_, predictors).sort_values(ascending=False)
    print(feat_imp)
    print(feat_imp.shape)
    # pred= lgb_model.predict(test[col])
    pred = lgb_model.predict_proba(test[col])[:, 1]
    test['pred'] = pred
    test['index'] = range(len(test))
    # print(test[['is_trade','pred']])
    print('误差 ', log_loss(test['is_trade'], test['pred']))
    print('KS ', ks_metric(test['is_trade'], test['pred']))
    ks = ks_metric(test['is_trade'], test['pred'])
    return best_iter,lgb_model,ks

In [4]:
def sub(train, test, best_iter):
    col = [c for c in train if
           c not in ['is_trade', 'item_category_list', 'item_property_list', 'predict_category_property', 'instance_id',
                     'context_id', 'realtime', 'context_timestamp']]
    X = train[col]
    y = train['is_trade'].values
    print('Training LGBM model...')
    lgb0 = lgb.LGBMClassifier(
        objective='binary',
        # metric='binary_error',
        num_leaves=35,
        depth=8,
        learning_rate=0.05,
        seed=2018,
        colsample_bytree=0.8,
        # min_child_samples=8,
        subsample=0.9,
        n_estimators=best_iter)
    lgb_model = lgb0.fit(X, y)
    predictors = [i for i in X.columns]
    feat_imp = pd.Series(lgb_model.feature_importances_, predictors).sort_values(ascending=False)
    print(feat_imp)
    print(feat_imp.shape)
    # pred= lgb_model.predict(test[col])
    pred = lgb_model.predict_proba(test[col])[:, 1]
    test['predicted_score'] = pred
    sub1 = test[['instance_id', 'predicted_score']]
    sub=pd.read_csv("../../Data/advertisment/Raw/round1_ijcai_18_test_a_20180301.txt", sep="\s+")
    sub=pd.merge(sub,sub1,on=['instance_id'],how='left')
    sub=sub.fillna(0)
    #sub[['instance_id', 'predicted_score']].to_csv('result/result0320.csv',index=None,sep=' ')
    #sub[['instance_id', 'predicted_score']].to_csv('../../Submission/advertisement/result401.txt',sep=" ",index=False)
    return sub[['instance_id', 'predicted_score']]

In [5]:
train = pd.read_csv("../../Data/advertisment/Raw/round1_ijcai_18_train_20180301.txt", sep="\s+")
test = pd.read_csv("../../Data/advertisment/Raw/round1_ijcai_18_test_a_20180301.txt", sep="\s+")
data = pd.concat([train, test])
data = data.drop_duplicates(subset='instance_id')  # 把instance id去重
print('make feature')
data = base_process(data)
data=shijian(data)
data=shop_fenduan(data)
data = slide_cnt(data)
data = zuhe(data)
print('----------------------------全局统计特征---------------------------------------------------')
data = item(data)
data = user(data)
data = user_item(data)
data = user_shop(data)
data=shop_item(data)

make feature
--------------------------------------------------------------item--------------------------------------------------------------
--------------------------------------------------------------user--------------------------------------------------------------
user 0,1 feature
--------------------------------------------------------------context--------------------------------------------------------------
context 0,1 feature
--------------------------------------------------------------shop--------------------------------------------------------------
3    493224
2      3146
1        25
Name: deliver_map, dtype: int64
3    490039
2      6243
1       113
Name: service_map, dtype: int64
3    394629
2     99788
1      1978
Name: de_map, dtype: int64
3    356965
2    138783
1       647
Name: review_map, dtype: int64
当前日期前一天的cnt
当前日期之前的cnt
前一个小时的统计量
item两两组合
user两两组合
shop两两组合
----------------------------全局统计特征---------------------------------------------------
一个item有多少brand,pric

In [13]:
features = ['is_trade']+['shop_score_delivery','item_id_user_prob','predict_category_property0','item_price_level','item_collected_level_salse_prob','user_id','sale_price','gender_star','user_star_level_user_age_prob','item_price_level_user_age_prob','len_item_property','hour_map','predict_category_property3','item_sales_level','user_cntx','item_category_list1','item_brand_id_user_prob','predict_category_property1','item_pv_level_salse_prob','user_age_level','shop_id_user_prob','item_id_shop_prob','shop_cntx','shop_score_service','sale_collect','predict_category_property2','item_property_list8','predict_category_property4','item_cntx','user_star_level_user_age_cnt','user_star_level_user_occ_prob','user_star_level','user_cnt1','item_sales_level_shop_rev_prob','item_price_level_user_prob','item_property_list9','item_collected_level_user_age_prob','item_cnt1','shop_review_positive_rate','shop_id','item_pv_level_brand_prob','item_sales_level_user_prob','item_pv_level_shop_prob','item_city_id_user_age_prob','shop_score_description','shop_cnt1','price_collect','item_sales_level_user_age_prob','item_id','item_price_level_shop_rev_prob','item_brand_id','shop_review_num_level_user_prob','item_city_id_user_prob','item_collected_level_item_prob','context_page_id','item_collected_level_salse_cnt','item_id_user_age_cnt','item_city_id_shop_rev_prob','item_property_list6','item_price_level_shop_cnt','user_star_level_user_occ_cnt','item_collected_level_city_prob','item_property_list3','item_property_list7','shop_id_user_age_cnt','item_pv_level_price_prob','item_pv_level_user_prob','item_price_level_city_prob','item_city_id_brand_prob','shop_review_num_level_user_gender_prob','shop_id_user_age_prob','item_id_user_age_prob','user_star_level_user_gender_prob','item_pv_level_coll_cnt','item_property_list4','item_id_user_gender_cnt','item_sales_level_item_prob','item_sales_level_price_prob','item_sales_level_user_age_cnt','item_pv_level_city_prob','len_predict_category_property','gender_age','item_sales_level_city_prob','item_sales_level_shop_rev_cnt','item_property_list2','item_collected_level_user_age_cnt','item_sales_level_user_occ_cnt','item_brand_id_user_age_cnt','item_pv_level_user_age_cnt','item_id_user_occ_cnt','item_id_user_occ_prob','shop_star_level_user_age_prob','item_property_list5','item_brand_id_user_occ_cnt','item_id_user_gender_prob','shop_star_level_user_prob','item_sales_level_shop_prob','item_pv_level_brand_cnt','item_collected_level_user_prob','item_price_level_brand_prob','item_price_level_shop_prob','item_collected_level_brand_cnt','shop_review_num_level_user_age_prob','item_collected_level_brand_prob','item_city_id_user_age_cnt','item_sales_level_city_cnt','shop_id_user_occ_cnt','user_star_level_user_gender_cnt','item_sales_level_brand_prob','item_collected_level_shop_rev_prob','item_id_shop_rev_prob','shop_star_level_user_gender_prob','item_brand_id_user_age_prob','user_gender_id_user_cnt','shop_review_num_level_user_occ_cnt','item_collected_level_price_prob','shop_review_num_level_user_gender_cnt','item_collected_level_shop_prob','item_sales_level_shop_cnt','item_price_level_user_age_cnt','item_pv_level_item_prob','item_collected_level_item_cnt','shop_review_num_level_user_age_cnt','item_collected_level_shop_rev_cnt','item_id_user_cnt','item_pv_level_city_cnt','user_occupation_id_user_cnt','shop_id_user_occ_prob','day','user_age_level_user_cnt','item_price_level_user_occ_cnt','item_collected_level_user_gender_cnt','item_brand_id_shop_rev_cnt','item_city_id_user_occ_cnt','item_city_id_user_occ_prob','user_occupation_id_user_age_cnt','item_sales_level_item_cnt','item_pv_level_coll_prob','shop_id_user_gender_cnt','item_sales_level_brand_cnt','item_pv_level_shop_cnt','item_pv_level_item_cnt','item_pv_level_user_age_prob','item_sales_level_user_occ_prob','item_pv_level_shop_rev_cnt','item_city_id_brand_cnt','item_price_level_shop_rev_cnt','item_brand_id_user_gender_prob','item_brand_id_user_gender_cnt','item_collected_level_user_cnt','item_city_id','item_collected_level_user_gender_prob','user_occupation_id_user_age_prob','item_city_id_user_gender_cnt','item_city_id_user_cnt','item_property_list1','shop_id_user_gender_prob','shop_star_level_user_gender_cnt','item_brand_id_shop_prob','item_city_id_user_gender_prob','item_pv_level_salse_cnt','item_pv_level_user_occ_cnt','item_collected_level_city_cnt','item_city_id_shop_rev_cnt','item_sales_level_user_gender_cnt','item_collected_level_user_occ_cnt','user_age_level_user_gender_prob','item_price_level_city_cnt','item_sales_level_user_cnt','shop_star_level_user_occ_prob','item_property_list0','item_collected_level_shop_cnt','item_city_id_shop_cnt','item_pv_level_shop_rev_prob','user_occupation_id_user_gender_cnt','item_brand_id_shop_rev_prob','shop_star_level_user_occ_cnt','shop_star_level_user_age_cnt','item_brand_id_user_occ_prob','item_brand_id_shop_cnt','item_price_level_user_cnt','shop_star_level','item_price_level_user_gender_prob','item_sales_level_user_gender_prob','item_pv_level_user_occ_prob','item_collected_level_user_occ_prob','shop_id_user_cnt','item_pv_level_user_gender_cnt','user_age_level_user_gender_cnt','item_pv_level_user_cnt','shop_review_num_level','item_brand_id_user_cnt','gender_occ','user_occupation_id_user_gender_prob','user_star_level_user_cnt','item_price_level_brand_cnt','item_sales_level_price_cnt','item_price_level_user_gender_cnt','item_price_level_user_occ_prob','item_collected_level','item_city_id_item_prob','shop_review_num_level_user_occ_prob','item_brand_id_item_cnt','shop_review_num_level_user_cnt','normal_shop','user_star_level_user_prob','shop_star_level_user_cnt','item_collected_level_price_cnt','item_pv_level_user_gender_prob','user_occupation_id','item_category_list2','item_pv_level','item_brand_id_item_prob','item_id_shop_rev_cnt','user_age_level_user_prob','item_price_level_item_cnt','item_city_id_shop_prob','user_gender_id_user_prob','context_page0','item_city_id_item_cnt','len_item_category','item_pv_level_price_cnt','item_price_level_item_prob']
train= data.loc[(data['day'] >= 18) & (data['day'] <= 23),features]
test= data.loc[(data['day'] == 24),features]

In [15]:
train= data[(data['day'] >= 18) & (data['day'] <= 23)]
test= data[(data['day'] == 24)]

In [15]:
del train['user_id']
del test['user_id']

In [6]:
tmp = train.loc[train['is_trade']==1]
train_new = pd.concat([train,tmp])

In [24]:
ksList = []
for leaf in range(5,21,5):
    best_iter,model,ks = lgbCV(train, test,leaf)
    ksList.append(ks)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.647801
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606755
[3]	valid_0's binary_logloss: 0.569466
[4]	valid_0's binary_logloss: 0.53544
[5]	valid_0's binary_logloss: 0.504286
[6]	valid_0's binary_logloss: 0.475681
[7]	valid_0's binary_logloss: 0.449347
[8]	valid_0's binary_logloss: 0.425063
[9]	valid_0's binary_logloss: 0.402621
[10]	valid_0's binary_logloss: 0.381826
[11]	valid_0's binary_logloss: 0.362549
[12]	valid_0's binary_logloss: 0.344628
[13]	valid_0's binary_logloss: 0.327963
[14]	valid_0's binary_logloss: 0.312446
[15]	valid_0's binary_logloss: 0.29797
[16]	valid_0's binary_logloss: 0.284469
[17]	valid_0's binary_logloss: 0.271869
[18]	valid_0's binary_logloss: 0.26008
[19]	valid_0's binary_logloss: 0.249064
[20]	valid_0's binary_logloss: 0.23875
[21]	valid_0's binary_logloss: 0.229091
[22]	valid_0's binary_logloss: 0.220046
[23]	valid_0's binary_logloss: 0.211569
[24]	va

[200]	valid_0's binary_logloss: 0.0815244
[201]	valid_0's binary_logloss: 0.0815212
[202]	valid_0's binary_logloss: 0.0815225
[203]	valid_0's binary_logloss: 0.0815228
[204]	valid_0's binary_logloss: 0.0815157
[205]	valid_0's binary_logloss: 0.0815102
[206]	valid_0's binary_logloss: 0.0815077
[207]	valid_0's binary_logloss: 0.0815033
[208]	valid_0's binary_logloss: 0.0814956
[209]	valid_0's binary_logloss: 0.0814904
[210]	valid_0's binary_logloss: 0.0814857
[211]	valid_0's binary_logloss: 0.0814799
[212]	valid_0's binary_logloss: 0.0814746
[213]	valid_0's binary_logloss: 0.0814725
[214]	valid_0's binary_logloss: 0.0814629
[215]	valid_0's binary_logloss: 0.0814584
[216]	valid_0's binary_logloss: 0.0814533
[217]	valid_0's binary_logloss: 0.0814548
[218]	valid_0's binary_logloss: 0.0814444
[219]	valid_0's binary_logloss: 0.0814358
[220]	valid_0's binary_logloss: 0.0814257
[221]	valid_0's binary_logloss: 0.0814252
[222]	valid_0's binary_logloss: 0.0814219
[223]	valid_0's binary_logloss: 0.

[396]	valid_0's binary_logloss: 0.081198
[397]	valid_0's binary_logloss: 0.0811813
[398]	valid_0's binary_logloss: 0.0811802
[399]	valid_0's binary_logloss: 0.0811797
[400]	valid_0's binary_logloss: 0.0811784
[401]	valid_0's binary_logloss: 0.0811747
[402]	valid_0's binary_logloss: 0.0811692
[403]	valid_0's binary_logloss: 0.0811669
[404]	valid_0's binary_logloss: 0.0811672
[405]	valid_0's binary_logloss: 0.0811599
[406]	valid_0's binary_logloss: 0.0811631
[407]	valid_0's binary_logloss: 0.0811637
[408]	valid_0's binary_logloss: 0.0811654
[409]	valid_0's binary_logloss: 0.0811607
[410]	valid_0's binary_logloss: 0.0811618
[411]	valid_0's binary_logloss: 0.0811625
[412]	valid_0's binary_logloss: 0.08116
[413]	valid_0's binary_logloss: 0.0811624
[414]	valid_0's binary_logloss: 0.0811652
[415]	valid_0's binary_logloss: 0.0811637
[416]	valid_0's binary_logloss: 0.0811631
[417]	valid_0's binary_logloss: 0.081164
[418]	valid_0's binary_logloss: 0.0811645
[419]	valid_0's binary_logloss: 0.0811

[592]	valid_0's binary_logloss: 0.0810816
[593]	valid_0's binary_logloss: 0.0810821
[594]	valid_0's binary_logloss: 0.0810836
[595]	valid_0's binary_logloss: 0.0810794
[596]	valid_0's binary_logloss: 0.0810822
[597]	valid_0's binary_logloss: 0.081083
[598]	valid_0's binary_logloss: 0.0810851
[599]	valid_0's binary_logloss: 0.0810835
[600]	valid_0's binary_logloss: 0.0810859
[601]	valid_0's binary_logloss: 0.0810905
[602]	valid_0's binary_logloss: 0.0810916
[603]	valid_0's binary_logloss: 0.0810919
[604]	valid_0's binary_logloss: 0.0810933
[605]	valid_0's binary_logloss: 0.081097
[606]	valid_0's binary_logloss: 0.0810923
[607]	valid_0's binary_logloss: 0.0810931
[608]	valid_0's binary_logloss: 0.0810913
[609]	valid_0's binary_logloss: 0.0810911
[610]	valid_0's binary_logloss: 0.0810934
[611]	valid_0's binary_logloss: 0.0810978
[612]	valid_0's binary_logloss: 0.0810968
[613]	valid_0's binary_logloss: 0.0810965
[614]	valid_0's binary_logloss: 0.081095
[615]	valid_0's binary_logloss: 0.081

[789]	valid_0's binary_logloss: 0.0810429
[790]	valid_0's binary_logloss: 0.0810433
[791]	valid_0's binary_logloss: 0.081044
[792]	valid_0's binary_logloss: 0.0810462
[793]	valid_0's binary_logloss: 0.0810382
[794]	valid_0's binary_logloss: 0.0810358
[795]	valid_0's binary_logloss: 0.0810396
[796]	valid_0's binary_logloss: 0.0810399
[797]	valid_0's binary_logloss: 0.0810409
[798]	valid_0's binary_logloss: 0.0810419
[799]	valid_0's binary_logloss: 0.0810473
[800]	valid_0's binary_logloss: 0.0810469
[801]	valid_0's binary_logloss: 0.0810457
[802]	valid_0's binary_logloss: 0.0810447
[803]	valid_0's binary_logloss: 0.0810441
[804]	valid_0's binary_logloss: 0.0810453
[805]	valid_0's binary_logloss: 0.081047
[806]	valid_0's binary_logloss: 0.0810481
[807]	valid_0's binary_logloss: 0.0810494
[808]	valid_0's binary_logloss: 0.0810487
[809]	valid_0's binary_logloss: 0.0810504
[810]	valid_0's binary_logloss: 0.0810503
[811]	valid_0's binary_logloss: 0.0810496
[812]	valid_0's binary_logloss: 0.08

...

[252]	valid_0's binary_logloss: 0.0810111
[253]	valid_0's binary_logloss: 0.0810128
[254]	valid_0's binary_logloss: 0.0810167
[255]	valid_0's binary_logloss: 0.0810169
[256]	valid_0's binary_logloss: 0.0810156
[257]	valid_0's binary_logloss: 0.081017
[258]	valid_0's binary_logloss: 0.0810264
[259]	valid_0's binary_logloss: 0.0810271
[260]	valid_0's binary_logloss: 0.081033
[261]	valid_0's binary_logloss: 0.0810381
[262]	valid_0's binary_logloss: 0.0810236
[263]	valid_0's binary_logloss: 0.0810144
[264]	valid_0's binary_logloss: 0.0810181
[265]	valid_0's binary_logloss: 0.0810043
[266]	valid_0's binary_logloss: 0.0810052
[267]	valid_0's binary_logloss: 0.0810091
[268]	valid_0's binary_logloss: 0.0810057
[269]	valid_0's binary_logloss: 0.0810077
[270]	valid_0's binary_logloss: 0.0810116
[271]	valid_0's binary_logloss: 0.0810115
[272]	valid_0's binary_logloss: 0.0810122
[273]	valid_0's binary_logloss: 0.0810136
[274]	valid_0's binary_logloss: 0.0810115
[275]	valid_0's binary_logloss: 0.08

[448]	valid_0's binary_logloss: 0.0811284
[449]	valid_0's binary_logloss: 0.0811307
[450]	valid_0's binary_logloss: 0.0811327
[451]	valid_0's binary_logloss: 0.0811347
[452]	valid_0's binary_logloss: 0.0811242
[453]	valid_0's binary_logloss: 0.0811221
[454]	valid_0's binary_logloss: 0.0811161
[455]	valid_0's binary_logloss: 0.0811229
[456]	valid_0's binary_logloss: 0.0811293
[457]	valid_0's binary_logloss: 0.0811319
[458]	valid_0's binary_logloss: 0.0811361
[459]	valid_0's binary_logloss: 0.0811427
[460]	valid_0's binary_logloss: 0.0811475
[461]	valid_0's binary_logloss: 0.0811473
[462]	valid_0's binary_logloss: 0.0811423
[463]	valid_0's binary_logloss: 0.081145
[464]	valid_0's binary_logloss: 0.0811495
[465]	valid_0's binary_logloss: 0.0811512
[466]	valid_0's binary_logloss: 0.0811495
[467]	valid_0's binary_logloss: 0.0811535
[468]	valid_0's binary_logloss: 0.081154
[469]	valid_0's binary_logloss: 0.0811257
[470]	valid_0's binary_logloss: 0.0811257
[471]	valid_0's binary_logloss: 0.08

[95]	valid_0's binary_logloss: 0.0827261
[96]	valid_0's binary_logloss: 0.082624
[97]	valid_0's binary_logloss: 0.0825587
[98]	valid_0's binary_logloss: 0.0825043
[99]	valid_0's binary_logloss: 0.0824244
[100]	valid_0's binary_logloss: 0.082358
[101]	valid_0's binary_logloss: 0.0823093
[102]	valid_0's binary_logloss: 0.0822472
[103]	valid_0's binary_logloss: 0.0821977
[104]	valid_0's binary_logloss: 0.0821414
[105]	valid_0's binary_logloss: 0.0821088
[106]	valid_0's binary_logloss: 0.0820653
[107]	valid_0's binary_logloss: 0.0820317
[108]	valid_0's binary_logloss: 0.0819978
[109]	valid_0's binary_logloss: 0.081957
[110]	valid_0's binary_logloss: 0.0819271
[111]	valid_0's binary_logloss: 0.081893
[112]	valid_0's binary_logloss: 0.0818698
[113]	valid_0's binary_logloss: 0.0818402
[114]	valid_0's binary_logloss: 0.0818038
[115]	valid_0's binary_logloss: 0.081779
[116]	valid_0's binary_logloss: 0.0817509
[117]	valid_0's binary_logloss: 0.0816877
[118]	valid_0's binary_logloss: 0.0816662
[1

[292]	valid_0's binary_logloss: 0.0808723
[293]	valid_0's binary_logloss: 0.0808708
[294]	valid_0's binary_logloss: 0.0808747
[295]	valid_0's binary_logloss: 0.0808853
[296]	valid_0's binary_logloss: 0.0808899
[297]	valid_0's binary_logloss: 0.0808964
[298]	valid_0's binary_logloss: 0.0808954
[299]	valid_0's binary_logloss: 0.0808956
[300]	valid_0's binary_logloss: 0.0808921
[301]	valid_0's binary_logloss: 0.0808992
[302]	valid_0's binary_logloss: 0.0808896
[303]	valid_0's binary_logloss: 0.0808919
[304]	valid_0's binary_logloss: 0.0808978
[305]	valid_0's binary_logloss: 0.0808993
[306]	valid_0's binary_logloss: 0.0809028
[307]	valid_0's binary_logloss: 0.0808904
[308]	valid_0's binary_logloss: 0.0808964
[309]	valid_0's binary_logloss: 0.0808864
[310]	valid_0's binary_logloss: 0.0808818
[311]	valid_0's binary_logloss: 0.0808911
[312]	valid_0's binary_logloss: 0.0808885
[313]	valid_0's binary_logloss: 0.0808907
[314]	valid_0's binary_logloss: 0.0808914
[315]	valid_0's binary_logloss: 0.

shop_score_delivery                      200
item_id_user_prob                        156
predict_category_property0               107
item_price_level                         104
user_id                                   97
item_collected_level_salse_prob           95
hour                                      95
sale_price                                95
item_price_level_user_age_prob            79
gender_star                               78
item_sales_level                          77
user_star_level_user_age_prob             75
item_brand_id_user_prob                   74
user_cntx                                 70
hour_map                                  68
item_category_list1                       62
len_item_property                         60
user_age_level                            57
predict_category_property3                55
shop_id_user_prob                         52
item_pv_level_salse_prob                  51
item_sales_level_shop_rev_prob            51
item_prope

In [13]:
best_iter,model = lgbCV(train_new, test)

Training LGBM model...
[1]	valid_0's binary_logloss: 0.653742
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.617818
[3]	valid_0's binary_logloss: 0.585038
[4]	valid_0's binary_logloss: 0.554987
[5]	valid_0's binary_logloss: 0.527282
[6]	valid_0's binary_logloss: 0.501785
[7]	valid_0's binary_logloss: 0.478267
[8]	valid_0's binary_logloss: 0.456473
[9]	valid_0's binary_logloss: 0.436255
[10]	valid_0's binary_logloss: 0.417433
[11]	valid_0's binary_logloss: 0.39994
[12]	valid_0's binary_logloss: 0.383621
[13]	valid_0's binary_logloss: 0.368371
[14]	valid_0's binary_logloss: 0.354126
[15]	valid_0's binary_logloss: 0.340846
[16]	valid_0's binary_logloss: 0.328414
[17]	valid_0's binary_logloss: 0.316812
[18]	valid_0's binary_logloss: 0.305859
[19]	valid_0's binary_logloss: 0.295645
[20]	valid_0's binary_logloss: 0.286054
[21]	valid_0's binary_logloss: 0.27704
[22]	valid_0's binary_logloss: 0.268533
[23]	valid_0's binary_logloss: 0.260589
[24]	

[202]	valid_0's binary_logloss: 0.120541
[203]	valid_0's binary_logloss: 0.120485
[204]	valid_0's binary_logloss: 0.120462
[205]	valid_0's binary_logloss: 0.12043
[206]	valid_0's binary_logloss: 0.120388
[207]	valid_0's binary_logloss: 0.120369
[208]	valid_0's binary_logloss: 0.120128
[209]	valid_0's binary_logloss: 0.120109
[210]	valid_0's binary_logloss: 0.120093
[211]	valid_0's binary_logloss: 0.120089
[212]	valid_0's binary_logloss: 0.120072
[213]	valid_0's binary_logloss: 0.120058
[214]	valid_0's binary_logloss: 0.120062
[215]	valid_0's binary_logloss: 0.120054
[216]	valid_0's binary_logloss: 0.120042
[217]	valid_0's binary_logloss: 0.12003
[218]	valid_0's binary_logloss: 0.12003
[219]	valid_0's binary_logloss: 0.120008
[220]	valid_0's binary_logloss: 0.119987
[221]	valid_0's binary_logloss: 0.119964
[222]	valid_0's binary_logloss: 0.119949
[223]	valid_0's binary_logloss: 0.119921
[224]	valid_0's binary_logloss: 0.119903
[225]	valid_0's binary_logloss: 0.119886
[226]	valid_0's bin

[403]	valid_0's binary_logloss: 0.115749
[404]	valid_0's binary_logloss: 0.11575
[405]	valid_0's binary_logloss: 0.115737
[406]	valid_0's binary_logloss: 0.11573
[407]	valid_0's binary_logloss: 0.115675
[408]	valid_0's binary_logloss: 0.115669
[409]	valid_0's binary_logloss: 0.115643
[410]	valid_0's binary_logloss: 0.115632
[411]	valid_0's binary_logloss: 0.115616
[412]	valid_0's binary_logloss: 0.115597
[413]	valid_0's binary_logloss: 0.115581
[414]	valid_0's binary_logloss: 0.115567
[415]	valid_0's binary_logloss: 0.115539
[416]	valid_0's binary_logloss: 0.11553
[417]	valid_0's binary_logloss: 0.115515
[418]	valid_0's binary_logloss: 0.115501
[419]	valid_0's binary_logloss: 0.115473
[420]	valid_0's binary_logloss: 0.115458
[421]	valid_0's binary_logloss: 0.115442
[422]	valid_0's binary_logloss: 0.115436
[423]	valid_0's binary_logloss: 0.115418
[424]	valid_0's binary_logloss: 0.115388
[425]	valid_0's binary_logloss: 0.11538
[426]	valid_0's binary_logloss: 0.115438
[427]	valid_0's bina

[604]	valid_0's binary_logloss: 0.111549
[605]	valid_0's binary_logloss: 0.111544
[606]	valid_0's binary_logloss: 0.111544
[607]	valid_0's binary_logloss: 0.11152
[608]	valid_0's binary_logloss: 0.111511
[609]	valid_0's binary_logloss: 0.111494
[610]	valid_0's binary_logloss: 0.11148
[611]	valid_0's binary_logloss: 0.11146
[612]	valid_0's binary_logloss: 0.11144
[613]	valid_0's binary_logloss: 0.111399
[614]	valid_0's binary_logloss: 0.111385
[615]	valid_0's binary_logloss: 0.111372
[616]	valid_0's binary_logloss: 0.111354
[617]	valid_0's binary_logloss: 0.111339
[618]	valid_0's binary_logloss: 0.111313
[619]	valid_0's binary_logloss: 0.111316
[620]	valid_0's binary_logloss: 0.111311
[621]	valid_0's binary_logloss: 0.111281
[622]	valid_0's binary_logloss: 0.111285
[623]	valid_0's binary_logloss: 0.111268
[624]	valid_0's binary_logloss: 0.111255
[625]	valid_0's binary_logloss: 0.111234
[626]	valid_0's binary_logloss: 0.111184
[627]	valid_0's binary_logloss: 0.111181
[628]	valid_0's bina

[805]	valid_0's binary_logloss: 0.108295
[806]	valid_0's binary_logloss: 0.108281
[807]	valid_0's binary_logloss: 0.108248
[808]	valid_0's binary_logloss: 0.108229
[809]	valid_0's binary_logloss: 0.108198
[810]	valid_0's binary_logloss: 0.108195
[811]	valid_0's binary_logloss: 0.108179
[812]	valid_0's binary_logloss: 0.108131
[813]	valid_0's binary_logloss: 0.108101
[814]	valid_0's binary_logloss: 0.108097
[815]	valid_0's binary_logloss: 0.108079
[816]	valid_0's binary_logloss: 0.108054
[817]	valid_0's binary_logloss: 0.108039
[818]	valid_0's binary_logloss: 0.108025
[819]	valid_0's binary_logloss: 0.10801
[820]	valid_0's binary_logloss: 0.108001
[821]	valid_0's binary_logloss: 0.108
[822]	valid_0's binary_logloss: 0.107973
[823]	valid_0's binary_logloss: 0.10795
[824]	valid_0's binary_logloss: 0.107931
[825]	valid_0's binary_logloss: 0.107903
[826]	valid_0's binary_logloss: 0.107897
[827]	valid_0's binary_logloss: 0.107878
[828]	valid_0's binary_logloss: 0.107864
[829]	valid_0's binar

...

[2935]	valid_0's binary_logloss: 0.0899485
[2936]	valid_0's binary_logloss: 0.0899402
[2937]	valid_0's binary_logloss: 0.0899342
[2938]	valid_0's binary_logloss: 0.0899323
[2939]	valid_0's binary_logloss: 0.0899293
[2940]	valid_0's binary_logloss: 0.0899281
[2941]	valid_0's binary_logloss: 0.0899337
[2942]	valid_0's binary_logloss: 0.0899356
[2943]	valid_0's binary_logloss: 0.0899352
[2944]	valid_0's binary_logloss: 0.0899233
[2945]	valid_0's binary_logloss: 0.0899166
[2946]	valid_0's binary_logloss: 0.0899165
[2947]	valid_0's binary_logloss: 0.0899118
[2948]	valid_0's binary_logloss: 0.0899058
[2949]	valid_0's binary_logloss: 0.0899064
[2950]	valid_0's binary_logloss: 0.0899012
[2951]	valid_0's binary_logloss: 0.0898977
[2952]	valid_0's binary_logloss: 0.0898992
[2953]	valid_0's binary_logloss: 0.0898979
[2954]	valid_0's binary_logloss: 0.0898966
[2955]	valid_0's binary_logloss: 0.089893
[2956]	valid_0's binary_logloss: 0.0898794
[2957]	valid_0's binary_logloss: 0.0898741
[2958]	valid

[3126]	valid_0's binary_logloss: 0.0895942
[3127]	valid_0's binary_logloss: 0.0895913
[3128]	valid_0's binary_logloss: 0.0895867
[3129]	valid_0's binary_logloss: 0.0895909
[3130]	valid_0's binary_logloss: 0.0895874
[3131]	valid_0's binary_logloss: 0.0895833
[3132]	valid_0's binary_logloss: 0.0895848
[3133]	valid_0's binary_logloss: 0.0895769
[3134]	valid_0's binary_logloss: 0.0895843
[3135]	valid_0's binary_logloss: 0.0895856
[3136]	valid_0's binary_logloss: 0.089586
[3137]	valid_0's binary_logloss: 0.0895871
[3138]	valid_0's binary_logloss: 0.0895806
[3139]	valid_0's binary_logloss: 0.0895736
[3140]	valid_0's binary_logloss: 0.0895519
[3141]	valid_0's binary_logloss: 0.0895427
[3142]	valid_0's binary_logloss: 0.0895358
[3143]	valid_0's binary_logloss: 0.0895323
[3144]	valid_0's binary_logloss: 0.0895293
[3145]	valid_0's binary_logloss: 0.0895312
[3146]	valid_0's binary_logloss: 0.0895205
[3147]	valid_0's binary_logloss: 0.0895165
[3148]	valid_0's binary_logloss: 0.0895166
[3149]	valid

[3319]	valid_0's binary_logloss: 0.0892946
[3320]	valid_0's binary_logloss: 0.0892985
[3321]	valid_0's binary_logloss: 0.0892999
[3322]	valid_0's binary_logloss: 0.0892976
[3323]	valid_0's binary_logloss: 0.0892946
[3324]	valid_0's binary_logloss: 0.0892914
[3325]	valid_0's binary_logloss: 0.0892883
[3326]	valid_0's binary_logloss: 0.0892832
[3327]	valid_0's binary_logloss: 0.0892923
[3328]	valid_0's binary_logloss: 0.089286
[3329]	valid_0's binary_logloss: 0.0892753
[3330]	valid_0's binary_logloss: 0.0892774
[3331]	valid_0's binary_logloss: 0.0892691
[3332]	valid_0's binary_logloss: 0.0892638
[3333]	valid_0's binary_logloss: 0.0892606
[3334]	valid_0's binary_logloss: 0.0892573
[3335]	valid_0's binary_logloss: 0.0892563
[3336]	valid_0's binary_logloss: 0.089255
[3337]	valid_0's binary_logloss: 0.089256
[3338]	valid_0's binary_logloss: 0.0892552
[3339]	valid_0's binary_logloss: 0.0892603
[3340]	valid_0's binary_logloss: 0.0892558
[3341]	valid_0's binary_logloss: 0.0892557
[3342]	valid_0

[3511]	valid_0's binary_logloss: 0.0890407
[3512]	valid_0's binary_logloss: 0.0890366
[3513]	valid_0's binary_logloss: 0.0890336
[3514]	valid_0's binary_logloss: 0.0890357
[3515]	valid_0's binary_logloss: 0.0890409
[3516]	valid_0's binary_logloss: 0.0890388
[3517]	valid_0's binary_logloss: 0.08904
[3518]	valid_0's binary_logloss: 0.0890409
[3519]	valid_0's binary_logloss: 0.0890398
[3520]	valid_0's binary_logloss: 0.0890358
[3521]	valid_0's binary_logloss: 0.0890386
[3522]	valid_0's binary_logloss: 0.0890396
[3523]	valid_0's binary_logloss: 0.0890355
[3524]	valid_0's binary_logloss: 0.0890348
[3525]	valid_0's binary_logloss: 0.0890348
[3526]	valid_0's binary_logloss: 0.089034
[3527]	valid_0's binary_logloss: 0.0890464
[3528]	valid_0's binary_logloss: 0.089049
[3529]	valid_0's binary_logloss: 0.0890477
[3530]	valid_0's binary_logloss: 0.08905
[3531]	valid_0's binary_logloss: 0.0890539
[3532]	valid_0's binary_logloss: 0.0890485
[3533]	valid_0's binary_logloss: 0.0890416
[3534]	valid_0's 

[3704]	valid_0's binary_logloss: 0.0890519
[3705]	valid_0's binary_logloss: 0.089053
[3706]	valid_0's binary_logloss: 0.0890542
[3707]	valid_0's binary_logloss: 0.0890501
[3708]	valid_0's binary_logloss: 0.0890466
[3709]	valid_0's binary_logloss: 0.0890392
[3710]	valid_0's binary_logloss: 0.0890388
[3711]	valid_0's binary_logloss: 0.0890371
[3712]	valid_0's binary_logloss: 0.0890396
[3713]	valid_0's binary_logloss: 0.0890415
[3714]	valid_0's binary_logloss: 0.0890404
[3715]	valid_0's binary_logloss: 0.0890427
[3716]	valid_0's binary_logloss: 0.089037
[3717]	valid_0's binary_logloss: 0.0890416
[3718]	valid_0's binary_logloss: 0.0890429
[3719]	valid_0's binary_logloss: 0.0890456
[3720]	valid_0's binary_logloss: 0.0890391
[3721]	valid_0's binary_logloss: 0.0890358
[3722]	valid_0's binary_logloss: 0.0890359
[3723]	valid_0's binary_logloss: 0.0890388
[3724]	valid_0's binary_logloss: 0.0890373
[3725]	valid_0's binary_logloss: 0.0890339
[3726]	valid_0's binary_logloss: 0.0890218
[3727]	valid_

In [25]:
ksList

[0.00030123684303788497,
 0.00014175851437076939,
 3.5439628592692347e-05,
 0.0007442322004465393]